In [1]:
import pandas as pd
import numpy as np
from IPython.display import display, HTML
from yf_utils import _2_split_train_val_test, _3_random_slices, _4_lookback_slices
from yf_utils import _5_perf_ranks, _6_grp_tuples_sort_sum
from myUtils import pickle_load, pickle_dump

pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 20)
pd.set_option('display.max_colwidth', 16)
pd.set_option('display.width', 800)

path_dir = "C:/Users/ping/MyDrive/stocks/yfinance/"
path_data_dump = path_dir + "VSCode_dump/"
fp_df_close_clean = 'df_close_clean'

# fp_df_eval_results = 'df_eval_results'
fp_df_eval_sym_freq_results = 'df_eval_sym_freq_results'

df_close_clean = pickle_load(path_data_dump, fp_df_close_clean)

In [2]:
# Split df_close_clean into training (df_train), validation (df_val) and test (df_test) set.
# The default split is 0.7, 0.2, 0.1 respectively.

###################################
# df_train, df_val, df_test = _2_split_train_val_test(df_close_clean, s_train=1, s_val=0, s_test=0)
df_train, df_val, df_test = _2_split_train_val_test(df_close_clean)
###################################

len_df_train = len(df_train)
len_df_val = len(df_val)
len_df_test = len(df_test)
print(f'len_df_train: {len_df_train}, len_df_val: {len_df_val}, len_df_test: {len_df_test} ')

len_df_train: 1050, len_df_val: 300, len_df_test: 150 


In [3]:
col_add0 = ['n_samples', 'days_lookbacks', 'days_eval', 'n_top_syms', 'sym_freq_cnt']
col_add1 = ['grp(retnStd/UI)_mean',   'grp(retnStd/UI)_std',   'grp(retnStd/UI)_mean/std']
col_add2 = ['grp(CAGR/retnStd)_mean', 'grp(CAGR/retnStd)_std', 'grp(CAGR/retnStd)_mean/std']
col_add3 = ['grp(CAGR/UI)_mean',      'grp(CAGR/UI)_std',      'grp(CAGR/UI)_mean/std']
col_add4 = ['SPY_retnStd/UI', 'SPY_CAGR/retnStd', 'SPY_CAGR/UI']
col_add_total = col_add0 + col_add1 + col_add2 + col_add3 + col_add4
print(f'col_add_total: {col_add_total}')

col_add_total: ['n_samples', 'days_lookbacks', 'days_eval', 'n_top_syms', 'sym_freq_cnt', 'grp(retnStd/UI)_mean', 'grp(retnStd/UI)_std', 'grp(retnStd/UI)_mean/std', 'grp(CAGR/retnStd)_mean', 'grp(CAGR/retnStd)_std', 'grp(CAGR/retnStd)_mean/std', 'grp(CAGR/UI)_mean', 'grp(CAGR/UI)_std', 'grp(CAGR/UI)_mean/std', 'SPY_retnStd/UI', 'SPY_CAGR/retnStd', 'SPY_CAGR/UI']


In [4]:
# # Creating Empty DataFrame and save it to file
# df = pd.DataFrame(columns=col_add_total)
# pickle_dump(df, path_data_dump, fp_df_eval_sym_freq_results)
# df

In [5]:
# if store_results:  # record results to df
# my_cols = ['n_samples', 'days_lookbacks', 'days_eval', 'n_top_syms', 'syms_start', 'syms_end', 'grp(CAGR/UI)_mean', 'grp(CAGR/UI)_std', 'grp(CAGR/UI)_mean/std', 'SPY_CAGR/UI']
# df_eval_results = pickle_load(path_data_dump, fp_df_eval_results)
df_eval_sym_freq_results = pickle_load(path_data_dump, fp_df_eval_sym_freq_results)

In [6]:
verbose = True  # True prints more output
# verbose = False  # True prints more output

# write run results to df_eval_results
# store_results = False
store_results = True

# number of max lookback tuples to create for iloc start_train:end_train:end_eval
# i.e. number of grp_top_set_syms_n_freq and grp_top_set_syms 
# n_samples = 400  
n_samples = 1

# for training, the number of days to lookback from iloc max-lookback end_train
# days_lookbacks = [15, 30, 60, 120]
# days_lookbacks = [30, 60, 120]
# days_lookbacks = [60, 120]
# days_lookbacks = [120]
# days_lookbacks = [60]
# days_lookbacks = [30]
days_lookbacks = [15]
# days_lookbacks = [15, 30]
# days_lookbacks = [15, 30, 60]
# days_lookbacks = [15, 29]
# days_lookbacks = [15, 31]
# days_lookbacks = [14, 30]
# days_lookbacks = [16, 32]
# days_lookbacks = [14, 28]

days_lookbacks.sort()

# number of days from end_train are used to evaluate effectiveness of the training
# days_eval = 10
days_eval = 5
# days_eval = 4
# days_eval = 3
# days_eval = 2  


# number of the most-common symbols from days_lookbacks' performance rankings to keep
n_top_syms = 20  

syms_start = 0  #  start index of n_top_syms for evaluation
# syms_start = 1  #  start index of n_top_syms for evaluation

# syms_end = n_top_syms  #  end index of n_top_syms for evaluation
# syms_end = 1  #  end index of n_top_syms for evaluation
# syms_end = 2  #  end index of n_top_syms for evaluation
# syms_end = 3  #  end index of n_top_syms for evaluation
# syms_end = 4  #  end index of n_top_syms for evaluation
# syms_end = 5  #  end index of n_top_syms for evaluation
# syms_end = 6  #  end index of n_top_syms for evaluation
syms_end = 10  #  end index of n_top_syms for evaluation

# freq_cnt = 6  # frequency of symbol in top_set_syms_n_freq 

create a sets of iloc lookback slices (start_train:end_train:end_eval), where  
end_train - start_train = days_lookback  
end_eval - end_train = days_eval  
for example,  
if given:  
 n_samples = 2  
 days_lookbacks = [30, 60, 120]  
 days_eval = 10  
a possible result is:  
 max_lookback_slices:  
 [(150, 270, 280), (5, 125, 135)]  
 where 270-150=125-5=max(days_lookbacks), 280-270=135-125=days_eval  
 sets_lookback_slices:  
 [[(240, 270, 280), (210, 270, 280), (150, 270, 280)], [(95, 125, 135), (65, 125, 135), (5, 125, 135)]]  
  where in a set, 270-240=days_lookbacks[0], 270-210=days_lookbacks[1], 270-150=days_lookbacks[2]  
  and 270, i.e. end_train, is constant for the set  

In [7]:
# return n_samples slices
max_lookback_slices = _3_random_slices(len_df_train, n_samples=n_samples, days_lookback=max(days_lookbacks), days_eval=days_eval)
# return n_samples * len(days_lookbacks) slices
sets_lookback_slices = _4_lookback_slices(max_slices=max_lookback_slices, days_lookbacks=days_lookbacks, verbose=False)

if verbose:
  print(f'max_lookback_slices:\n{max_lookback_slices}')
  print(f'sets_lookback_slices:\n{sets_lookback_slices}')

max_lookback_slices:
[(1024, 1039, 1044)]
sets_lookback_slices:
[[(1024, 1039, 1044)]]


Generate lists, n_samples long, of the highest performance ranked symbols. The performance metrics are: CAGR/UI, CAGR/retnStd, retnStd/UI. n_top_syms of the best performing symbols from each metric are combined. The symbols are sorted by their number of appearances in the combined pool, and are placed in a list. A slice of the best performing symbols is selected by syms_start:syms_end, i.e. top_set_syms_n_freq[syms_start:syms_end].     

The performance metrics are calculated based on slices in sets_lookback_slices.  The first two numbers are ilocs for training. The last two numbers are ilocs for evaluation.

In [8]:
grp_top_set_syms_n_freq = []  # list of lists of top_set_symbols_n_freq, there are n_samples lists in list
grp_top_set_syms = []  # list of lists of top_set_symbols, there are n_samples lists in list
# loop thru lists of tuples of start_train:end_train:end_eval, i.e.
#  [[(887, 917, 927), (857, 917, 927), (797, 917, 927)],
#  [(483, 513, 523), (453, 513, 523), (393, 513, 523)]]
for lb_slices in sets_lookback_slices:
  grp_most_common_syms = []  
  for lb_slice in lb_slices:  # lb_slice, e.g. (246, 276, 286)
    start_train = lb_slice[0]
    end_train = lb_slice[1]
    start_eval = end_train
    end_eval = lb_slice[2]
    lookback = end_train - start_train
    d_eval = end_eval - start_eval

    if verbose:
      print(f'days lookback: {lookback}')
      print(f'lb_slices:     {lb_slices}')
      print(f'lb_slice:      {lb_slice}')
      print(f'days eval:     {d_eval}')    
      print(f'start_train:   {start_train}')
      print(f'end_train:     {end_train}')
      # print(f'start_eval:    {start_eval}')
      # print(f'end_eval:      {end_eval}')`

    _df = df_train.iloc[start_train:end_train]
    perf_ranks, most_common_syms = _5_perf_ranks(_df, n_top_syms=n_top_syms)
    grp_most_common_syms.append(most_common_syms)
    
    if verbose:    
      # 1 lookback of r_CAGR/UI, r_CAGR/retnStd, r_retnStd/UI
      print(f'perf_ranks: {perf_ranks}')  
      # most common symbols of perf_ranks 
      print(f'most_common_syms: {most_common_syms}')     
      # grp_perf_ranks[lookback] = perf_ranks
      print(f'+++ finish lookback slice {lookback} +++\n')

  if verbose:
    print(f'grp_most_common_syms: {grp_most_common_syms}')
    # grp_most_common_syms a is list of lists of tuples of 
    #  the most-common-symbols symbol:frequency cumulated from
    #  each days_lookback  
    print(f'**** finish lookback slices {lb_slices} ****\n')

  # flatten list of lists of (symbol:frequency)
  flat_grp_most_common_syms = [val for sublist in grp_most_common_syms for val in sublist]
  # group symbols from set of days_lookbacks (i.e. lb_slices) and sum frequency of the symbols
  set_most_common_syms = _6_grp_tuples_sort_sum(flat_grp_most_common_syms, reverse=True)
  # get the top few most-frequent symbol:frequency pairs
  top_set_syms_n_freq = set_most_common_syms[0:n_top_syms]
  # get symbols from top_set_syms_n_freq

###################################  
  # top_set_syms = [i[0] for i in top_set_syms_n_freq]
  top_set_syms = [i[0] for i in top_set_syms_n_freq[syms_start:syms_end]]  
###################################  
  
  grp_top_set_syms_n_freq.append(top_set_syms_n_freq)
  grp_top_set_syms.append(top_set_syms)

  if verbose:  
    print(f'top {n_top_syms} ranked symbols and frequency from set {lb_slices}:\n{top_set_syms_n_freq}')
    print(f'top {n_top_syms} ranked symbols from set {lb_slices}:\n{top_set_syms}')  
    print(f'===== finish top {n_top_syms} ranked symbols from days_lookback set {lb_slices} =====\n\n')

days lookback: 15
lb_slices:     [(1024, 1039, 1044)]
lb_slice:      (1024, 1039, 1044)
days eval:     5
start_train:   1024
end_train:     1039
perf_ranks: {'period-15': {'r_CAGR/UI': array(['NVCR', 'RFP', 'AIZ', 'AEO', 'ISRG', 'GMED', 'MAN', 'STAA', 'DOX',
       'CHRD', 'AEM', 'IQV', 'CARA', 'FPI', 'NSA', 'EDU', 'GOLD', 'AXS',
       'HGV', 'BRO'], dtype=object), 'r_CAGR/retnStd': array(['NVCR', 'RFP', 'AEO', 'CHRD', 'CARA', 'STAA', 'GMED', 'MAG', 'MAN',
       'ISRG', 'FPI', 'AEM', 'CATO', 'VNOM', 'HALO', 'CTRN', 'AIZ', 'IQV',
       'SBLK', 'SSRM'], dtype=object), 'r_retnStd/UI': array(['AIZ', 'DOX', 'ISRG', 'NAD', 'NVCR', 'NEA', 'D', 'MAN', 'GMED',
       'SKM', 'WSO', 'ORI', 'BSX', 'BRO', 'SYK', 'AXS', 'NSA', 'FIS',
       'BXP', 'POR'], dtype=object)}}
most_common_syms: [('NVCR', 3), ('AIZ', 3), ('ISRG', 3), ('GMED', 3), ('MAN', 3), ('RFP', 2), ('AEO', 2), ('STAA', 2), ('DOX', 2), ('CHRD', 2), ('AEM', 2), ('IQV', 2), ('CARA', 2), ('FPI', 2), ('NSA', 2), ('AXS', 2), ('BRO', 2), 

In [9]:
def top_set_sym_freq_cnt(top_set_syms_n_freq):
    sym_freq_cnt_6 = []
    sym_freq_cnt_5 = []
    sym_freq_cnt_4 = []
    sym_freq_cnt_3 = []
    sym_freq_cnt_2 = []

    for sym_n_freq in top_set_syms_n_freq:
        _sym = sym_n_freq[0]
        _freq = sym_n_freq[1]
        # print(_sym, _freq)
        if _freq == 6:
            sym_freq_cnt_6.append(_sym)
        elif _freq == 5:
            sym_freq_cnt_5.append(_sym)
        elif _freq == 4:
            sym_freq_cnt_4.append(_sym)
        elif _freq == 3:
            sym_freq_cnt_3.append(_sym)          
        else:
            sym_freq_cnt_2.append(_sym)

    l_sym_freq_cnt = []
    l_sym_freq_cnt.append(sym_freq_cnt_6)
    l_sym_freq_cnt.append(sym_freq_cnt_5)
    l_sym_freq_cnt.append(sym_freq_cnt_4)
    l_sym_freq_cnt.append(sym_freq_cnt_3)    
    l_sym_freq_cnt.append(sym_freq_cnt_2)    

    # if verbose:
    #     print(f'sym_freq_cnt_6: {sym_freq_cnt_6}')
    #     print(f'sym_freq_cnt_5: {sym_freq_cnt_5}')
    #     print(f'sym_freq_cnt_4: {sym_freq_cnt_4}')
    #     print(f'sym_freq_cnt_3: {sym_freq_cnt_3}')
    #     print(f'sym_freq_cnt_2: {sym_freq_cnt_2}')

    # return sym_freq_cnt_6, sym_freq_cnt_5, sym_freq_cnt_4, sym_freq_cnt_3, sym_freq_cnt_2
    return l_sym_freq_cnt    

In [10]:
# for top_set_syms_n_freq in grp_top_set_syms_n_freq:
#   l_sym_freq_cnt = top_set_sym_freq_cnt(top_set_syms_n_freq)
#   print(f'sym_freq_6: {l_sym_freq_cnt[0]}')
#   print(f'sym_freq_5: {l_sym_freq_cnt[1]}')
#   print(f'sym_freq_4: {l_sym_freq_cnt[2]}')
#   print(f'sym_freq_3: {l_sym_freq_cnt[3]}')
#   print(f'sym_freq_2: {l_sym_freq_cnt[4]}\n')  

In [11]:
from yf_utils import _7_perf_eval

print('z_grp_top_set_syms_n_freq:')
z_grp_top_set_syms_n_freq = zip(max_lookback_slices, grp_top_set_syms_n_freq)

for i, (_lookback_slice, _top_set_syms_n_freq) in enumerate(z_grp_top_set_syms_n_freq):

  start_train = _lookback_slice[0]
  end_train = _lookback_slice[1]
  start_eval = end_train
  end_eval = _lookback_slice[2]

  print(f'{i + 1 } of {n_samples} max_lookback_slice')
  print(f'max_lookback_slice: {_lookback_slice}')
  # dates correspond to max_lookback_slice
  d_start_train = df_train.index[start_train].strftime('%Y-%m-%d')
  d_end_train = df_train.index[end_train].strftime('%Y-%m-%d')
  d_start_eval = d_end_train
  d_end_eval = df_train.index[end_eval].strftime('%Y-%m-%d')
  d_df_eval_start = d_end_train
  d_df_eval_end = df_train.index[end_eval - 1].strftime('%Y-%m-%d')  
  print(f'max lookback dates: {d_start_train}, {d_end_train}, {d_end_eval}')
  print(f'df_eval dates (inclusive): {d_df_eval_start} - {d_df_eval_end}')    
  print(f'top_set_syms_n_freq: {_top_set_syms_n_freq}\n')

  l_sym_freq_cnt = top_set_sym_freq_cnt(_top_set_syms_n_freq)
  print(f'sym_freq_6: {l_sym_freq_cnt[0]}')
  print(f'sym_freq_5: {l_sym_freq_cnt[1]}')
  print(f'sym_freq_4: {l_sym_freq_cnt[2]}')
  print(f'sym_freq_3: {l_sym_freq_cnt[3]}\n')

  _sym_idx = ['SPY']
  df_SPY = df_train[start_eval:end_eval][_sym_idx]
  _, SPY_retnStd_d_UI, SPY_CAGR_d_retnStd, SPY_CAGR_d_UI = _7_perf_eval(df_SPY)
  print(f'\nSPY: retnStd/UI, CAGR/retnStd, CAGR/UI: {SPY_retnStd_d_UI[0]:>13,.3f}, {SPY_CAGR_d_retnStd[0]:>13,.3f}, {SPY_CAGR_d_UI[0]:>13,.3f}')

  # drop last list in l_sym_freq_cnt from zip
  zip_cnt_n_syms = zip([6,5,4,3], l_sym_freq_cnt[:-1])  
  for item in zip_cnt_n_syms:
    sym_freq_cnt = item[0]
    syms = item[1]
    if syms:  # iterate ONLY if there are symbols in syms
      df_eval = df_train[start_eval:end_eval][syms]      

      if verbose:
        # print(f'start_eval: {start_eval}')
        # print(f'end_eval:   {end_eval}')  
        print(f'start_eval: {start_eval},  date: {d_end_train}')
        print(f'end_eval:   {end_eval},  date: {d_end_eval},  df_eval last date: {d_df_eval_end}')
        print(f'frequency count of symbol(s): {sym_freq_cnt}')      
        # print(f'\ndf_eval:\n{df_eval}\n')

        syms_n_SPY = syms + ['SPY']
        df_eval_n_SPY = df_train[start_eval:end_eval][syms_n_SPY]   
        print(f'\ndf_eval_n_SPY:\n{df_eval_n_SPY}\n')


      _, grp_retnStd_d_UI, grp_CAGR_d_retnStd, grp_CAGR_d_UI = _7_perf_eval(df_eval)
      print(f'grp(retnStd/UI):   mean, std, mean/std: {grp_retnStd_d_UI[0]  :>13,.3f}, {grp_retnStd_d_UI[1]  :>13,.3f}, {grp_retnStd_d_UI[2]  :>13,.3f}')
      print(f'grp(CAGR/retnStd): mean, std, mean/std: {grp_CAGR_d_retnStd[0]:>13,.3f}, {grp_CAGR_d_retnStd[1]:>13,.3f}, {grp_CAGR_d_retnStd[2]:>13,.3f}')
      print(f'grp(CAGR/UI):      mean, std, mean/std: {grp_CAGR_d_UI[0]     :>13,.3f}, {grp_CAGR_d_UI[1]     :>13,.3f}, {grp_CAGR_d_UI[2]     :>13,.3f}')

      if store_results:  # record results to df
        # row_add = [n_samples, str(days_lookbacks), days_eval, n_top_syms, syms_start, syms_end, grp_CAGR_d_UI[0], grp_CAGR_d_UI[1], grp_CAGR_d_UI[2], SPY_CAGR_d_UI[0]]
        # df_eval_results.loc[len(df_eval_results)] = row_add
        # row_add0      = [n_samples, str(days_lookbacks), days_eval, n_top_syms, syms_start, syms_end]
        row_add0      = [n_samples, str(days_lookbacks), days_eval, n_top_syms, sym_freq_cnt]        
        row_add1      = [grp_retnStd_d_UI[0],   grp_retnStd_d_UI[1],   grp_retnStd_d_UI[2]]
        row_add2      = [grp_CAGR_d_retnStd[0], grp_CAGR_d_retnStd[1], grp_CAGR_d_retnStd[2]]
        row_add3      = [grp_CAGR_d_UI[0],      grp_CAGR_d_UI[1],      grp_CAGR_d_UI[2]]
        row_add4      = [SPY_retnStd_d_UI[0],   SPY_CAGR_d_retnStd[0], SPY_CAGR_d_UI[0]]
        row_add_total = row_add0 + row_add1 + row_add2 + row_add3 + row_add4
        print(f'row_add_total: {row_add_total}')
        df_eval_sym_freq_results.loc[len(df_eval_sym_freq_results)] = row_add_total
        # print(f'appended row_add to df_eval_results:\n{row_add}\n')
        print(f'appended row_add to df_eval_sym_freq_results:\n{row_add_total}\n')
  
  print('='*50, '\n')  

z_grp_top_set_syms_n_freq:
1 of 1 max_lookback_slice
max_lookback_slice: (1024, 1039, 1044)
max lookback dates: 2021-03-31, 2021-04-22, 2021-04-29
df_eval dates (inclusive): 2021-04-22 - 2021-04-28
top_set_syms_n_freq: [('AIZ', 3), ('GMED', 3), ('ISRG', 3), ('MAN', 3), ('NVCR', 3), ('AEM', 2), ('AEO', 2), ('AXS', 2), ('BRO', 2), ('CARA', 2), ('CHRD', 2), ('DOX', 2), ('FPI', 2), ('IQV', 2), ('NSA', 2), ('RFP', 2), ('STAA', 2), ('BSX', 1), ('BXP', 1), ('CATO', 1)]

sym_freq_6: []
sym_freq_5: []
sym_freq_4: []
sym_freq_3: ['AIZ', 'GMED', 'ISRG', 'MAN', 'NVCR']


SPY: retnStd/UI, CAGR/retnStd, CAGR/UI:        21.555,       223.653,     4,820.757
start_eval: 1039,  date: 2021-04-22
end_eval:   1044,  date: 2021-04-29,  df_eval last date: 2021-04-28
frequency count of symbol(s): 3

df_eval_n_SPY:
                   AIZ       GMED        ISRG         MAN        NVCR         SPY
Date                                                                             
2021-04-22  149.325333  71.010002 

In [41]:
df = df_eval_n_SPY.copy()
df

,AIZ,GMED,ISRG,MAN,NVCR,SPY
Date,,,,,,
2021-04-22,149.325333,71.010002,286.553345,114.288567,199.199997,410.925629
2021-04-23,151.208282,71.120003,291.843323,116.973694,207.630005,415.381042
2021-04-26,150.606506,71.500000,288.646667,116.348793,217.649994,416.248199
2021-04-27,151.742096,71.300003,287.126678,118.311363,212.500000,416.158508
2021-04-28,150.043564,71.739998,289.596680,119.902916,205.500000,416.038879


In [43]:
# pickle_dump(df, path_data_dump, 'df_eval_n_SPY_test_data')
_df = pickle_load(path_data_dump, 'df_eval_n_SPY_test_data')
_df

,AIZ,GMED,ISRG,MAN,NVCR,SPY
Date,,,,,,
2021-04-22,149.325333,71.010002,286.553345,114.288567,199.199997,410.925629
2021-04-23,151.208282,71.120003,291.843323,116.973694,207.630005,415.381042
2021-04-26,150.606506,71.500000,288.646667,116.348793,217.649994,416.248199
2021-04-27,151.742096,71.300003,287.126678,118.311363,212.500000,416.158508
2021-04-28,150.043564,71.739998,289.596680,119.902916,205.500000,416.038879


In [13]:
from myUtils import symb_perf_stats_vectorized_v7
(
  symbols,
  period_yr,
  retn,
  DD,
  UI,
  MDD,
  retnMean,
  retnStd,
  retnStd_div_UI,
  CAGR,
  CAGR_div_retnStd,
  CAGR_div_UI,
  grp_MeanCAGR,
  grp_StdCAGR,    
) = symb_perf_stats_vectorized_v7(df)

In [34]:
print(f'symbols: {symbols}')
print(f'period_yr: {period_yr}')
print(f'returns:\n{retn}')
print(f'drawdown:\n{DD}')
print(f'UI:\n{UI}')
print(f'max_drawdown:\n{MDD}')
print(f'returns_mean:\n{retnMean}')
print(f'returns_std:\n{retnStd}')
print(f'returns_std_div_UI:\n{retnStd_div_UI}')
print(f'CAGR:\n{CAGR}')
print(f'CAGR_div_returns_std:\n{CAGR_div_retnStd}')
print(f'CAGR_div_UI:\n{CAGR_div_UI}')
print(f'grp_mean_CAGR: {grp_MeanCAGR:>12.6f}')
print(f'grp_std_CAGR:  {grp_StdCAGR:>12.6f}')

symbols: Index(['AIZ', 'GMED', 'ISRG', 'MAN', 'NVCR', 'SPY'], dtype='object')
period_yr: 0.015873015873015872
returns:
                 AIZ      GMED      ISRG       MAN      NVCR       SPY
Date                                                                  
2021-04-22  0.000000  0.000000  0.000000  0.000000  0.000000  0.000000
2021-04-23  0.012610  0.001549  0.018461  0.023494  0.042319  0.010842
2021-04-26 -0.003980  0.005343 -0.010953 -0.005342  0.048259  0.002088
2021-04-27  0.007540 -0.002797 -0.005266  0.016868 -0.023662 -0.000215
2021-04-28 -0.011194  0.006171  0.008602  0.013452 -0.032941 -0.000287
drawdown:
                 AIZ      GMED      ISRG       MAN      NVCR       SPY
Date                                                                  
2021-04-22  0.000000  0.000000  0.000000  0.000000  0.000000  0.000000
2021-04-23  0.000000  0.000000  0.000000  0.000000  0.000000  0.000000
2021-04-26 -0.003980  0.000000 -0.010953 -0.005342  0.000000  0.000000
2021-04-27  0.00000

In [30]:
df_eval_sym_freq_results.columns

Index(['n_samples', 'days_lookbacks', 'days_eval', 'n_top_syms', 'sym_freq_cnt', 'grp(retnStd/UI)_mean', 'grp(retnStd/UI)_std', 'grp(retnStd/UI)_mean/std', 'grp(CAGR/retnStd)_mean', 'grp(CAGR/retnStd)_std', 'grp(CAGR/retnStd)_mean/std', 'grp(CAGR/UI)_mean', 'grp(CAGR/UI)_std', 'grp(CAGR/UI)_mean/std', 'SPY_retnStd/UI', 'SPY_CAGR/retnStd', 'SPY_CAGR/UI'], dtype='object')

In [31]:
row_add_total

[1,
 '[15]',
 5,
 20,
 3,
 2.6973254960720663,
 1.5657659371814052,
 1.7226875563071862,
 409.1099026076334,
 657.6121844568814,
 0.6221142373533045,
 1856.9232429558367,
 3538.2600202308017,
 0.5248125441144682,
 21.55465636483791,
 223.6527081672364,
 4820.757269610157]

In [15]:
df_eval_sym_freq_results

,n_samples,days_lookbacks,days_eval,n_top_syms,sym_freq_cnt,grp(retnStd/UI)_mean,grp(retnStd/UI)_std,grp(retnStd/UI)_mean/std,grp(CAGR/retnStd)_mean,grp(CAGR/retnStd)_std,grp(CAGR/retnStd)_mean/std,grp(CAGR/UI)_mean,grp(CAGR/UI)_std,grp(CAGR/UI)_mean/std,SPY_retnStd/UI,SPY_CAGR/retnStd,SPY_CAGR/UI
0,400,"[15, 30, 60,...",5,20,6,0.883658,0.375847,2.351113,-0.040546,1.496899e-02,-2.708658,-3.301565e-02,0.002012,-16.412918,0.617894,-0.051131,-0.031593
1,400,"[15, 30, 60,...",5,20,5,1.353213,1.293785,1.045933,-0.019376,4.706790e-02,-0.411656,4.228344e-03,0.038625,0.109472,0.617894,-0.051131,-0.031593
2,400,"[15, 30, 60,...",5,20,4,0.782797,0.293927,2.663239,-0.042616,1.672050e-02,-2.548702,-2.926511e-02,0.005271,-5.552463,0.617894,-0.051131,-0.031593
3,400,"[15, 30, 60,...",5,20,3,2.778897,2.417541,1.149473,-0.001122,3.090350e-02,-0.036299,5.048031e-02,0.109790,0.459791,0.617894,-0.051131,-0.031593
4,400,"[15, 30, 60,...",5,20,6,0.727441,0.505301,1.439618,-0.053275,4.010657e-02,-1.328344,-2.335990e-02,0.005917,-3.948001,0.867602,-0.013048,-0.011321
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10459,1,[15],5,20,3,inf,NaN,NaN,0.019698,5.221355e-02,0.377265,inf,NaN,NaN,8.866844,0.057082,0.506138
10460,1,[15],5,20,3,inf,NaN,NaN,0.046946,6.800600e-02,0.690323,inf,NaN,NaN,1.509084,-0.006903,-0.010417
10461,1,[15],5,20,3,inf,NaN,NaN,872062.083888,2.133847e+06,0.408681,inf,NaN,NaN,0.596112,-149.161024,-88.916732
10462,1,[15],5,20,3,3.227244,3.698447,0.872595,10406.894720,2.217039e+04,0.469405,9.646354e+04,211465.732873,0.456166,3.625955,266.898551,967.762075


In [16]:
# # Creating Empty DataFrame and Storing it in variable df
# df_eval_results = pd.DataFrame(columns=col_add_total)
pickle_dump(df_eval_sym_freq_results, path_data_dump, fp_df_eval_sym_freq_results)
df = pickle_load(path_data_dump, fp_df_eval_sym_freq_results)
df

,n_samples,days_lookbacks,days_eval,n_top_syms,sym_freq_cnt,grp(retnStd/UI)_mean,grp(retnStd/UI)_std,grp(retnStd/UI)_mean/std,grp(CAGR/retnStd)_mean,grp(CAGR/retnStd)_std,grp(CAGR/retnStd)_mean/std,grp(CAGR/UI)_mean,grp(CAGR/UI)_std,grp(CAGR/UI)_mean/std,SPY_retnStd/UI,SPY_CAGR/retnStd,SPY_CAGR/UI
0,400,"[15, 30, 60,...",5,20,6,0.883658,0.375847,2.351113,-0.040546,1.496899e-02,-2.708658,-3.301565e-02,0.002012,-16.412918,0.617894,-0.051131,-0.031593
1,400,"[15, 30, 60,...",5,20,5,1.353213,1.293785,1.045933,-0.019376,4.706790e-02,-0.411656,4.228344e-03,0.038625,0.109472,0.617894,-0.051131,-0.031593
2,400,"[15, 30, 60,...",5,20,4,0.782797,0.293927,2.663239,-0.042616,1.672050e-02,-2.548702,-2.926511e-02,0.005271,-5.552463,0.617894,-0.051131,-0.031593
3,400,"[15, 30, 60,...",5,20,3,2.778897,2.417541,1.149473,-0.001122,3.090350e-02,-0.036299,5.048031e-02,0.109790,0.459791,0.617894,-0.051131,-0.031593
4,400,"[15, 30, 60,...",5,20,6,0.727441,0.505301,1.439618,-0.053275,4.010657e-02,-1.328344,-2.335990e-02,0.005917,-3.948001,0.867602,-0.013048,-0.011321
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10459,1,[15],5,20,3,inf,NaN,NaN,0.019698,5.221355e-02,0.377265,inf,NaN,NaN,8.866844,0.057082,0.506138
10460,1,[15],5,20,3,inf,NaN,NaN,0.046946,6.800600e-02,0.690323,inf,NaN,NaN,1.509084,-0.006903,-0.010417
10461,1,[15],5,20,3,inf,NaN,NaN,872062.083888,2.133847e+06,0.408681,inf,NaN,NaN,0.596112,-149.161024,-88.916732
10462,1,[15],5,20,3,3.227244,3.698447,0.872595,10406.894720,2.217039e+04,0.469405,9.646354e+04,211465.732873,0.456166,3.625955,266.898551,967.762075


In [17]:
# https://towardsdatascience.com/pandas-groupby-a-simple-but-detailed-tutorial-314b8f37005d
# https://towardsdatascience.com/accessing-data-in-a-multiindex-dataframe-in-pandas-569e8767201d
# https://towardsdatascience.com/summarizing-data-with-pandas-crosstab-efc8b9abecf
# https://towardsdatascience.com/how-to-flatten-multiindex-columns-and-rows-in-pandas-f5406c50e569
# https://datascientyst.com/list-aggregation-functions-aggfunc-groupby-pandas/

In [18]:
print(f'len(df.columns): {len(df.columns)}')
print(f'df.columns: {df.columns}')

len(df.columns): 17
df.columns: Index(['n_samples', 'days_lookbacks', 'days_eval', 'n_top_syms', 'sym_freq_cnt', 'grp(retnStd/UI)_mean', 'grp(retnStd/UI)_std', 'grp(retnStd/UI)_mean/std', 'grp(CAGR/retnStd)_mean', 'grp(CAGR/retnStd)_std', 'grp(CAGR/retnStd)_mean/std', 'grp(CAGR/UI)_mean', 'grp(CAGR/UI)_std', 'grp(CAGR/UI)_mean/std', 'SPY_retnStd/UI', 'SPY_CAGR/retnStd', 'SPY_CAGR/UI'], dtype='object')


In [19]:
# pd.crosstab(df['days_lookbacks'], df['grp(CAGR/UI)_mean'])
tbl = df.groupby(['days_lookbacks', 'days_eval', 'sym_freq_cnt'])\
        .agg({'grp(CAGR/retnStd)_mean':     ['mean', 'std'],
              'grp(CAGR/retnStd)_mean/std': ['mean', 'std'],
              'SPY_CAGR/retnStd':           ['mean', 'std']})

In [20]:
tbl

grp(CAGR/retnStd)_mean               grp(CAGR/retnStd)_mean/std             SPY_CAGR/retnStd           
                                                          mean           std                       mean         std             mean        std
days_lookbacks   days_eval sym_freq_cnt                                                                                                        
[120]            5         3                    0.012459            0.029804         0.364854              1.217032         0.017158   0.050010
[15, 30, 60, ... 5         3                    0.004119            0.034305        -0.090480              2.128125         0.008163   0.051853
                           4                    0.003047            0.028861         0.169792              2.509705         0.009486   0.050949
                           5                    0.006333            0.052436        -0.001727              1.659893         0.009541   0.050911
                           6                    0.004989            0.042154        -1.498402             36.746818         0.007686   0.049116
[15, 30, 60]     5         3                    0.006098            0.028129         0.119825              0.713035         0.015308   0.047964
                           4                    0.004900            0.033411         0.575942              8.536839         0.015292   0.047954
                           5                    0.007569            0.040046         1.149887             18.025109         0.014831   0.047090
                           6                    0.007009            0.043564         0.171435              2.710300         0.015794   0.048252
[15, 30]         5         3                    0.006109            0.024416         0.142637              0.616667         0.015046   0.047532
                           4                    0.006659            0.037435         1.376197             23.283218         0.015032   0.048113
                           5                    0.006582            0.039122         0.305097              2.552450         0.013390   0.046803
                           6                    0.003026            0.040515        -0.464578              6.790572         0.015675   0.044950
[15]             5         3                 2034.288770        41862.104913         0.200269              0.732661         0.802652  18.187234
[30, 60, 120]    5         3                    0.007151            0.027594         0.181124              1.796506         0.017200   0.049004
                           4                    0.005190            0.028486         0.176843              1.127394         0.016957   0.049012
                           5                    0.018160            0.398843         3.912322            101.978124         0.016680   0.048727
                           6                    0.008071            0.046032         1.071601              9.622734         0.018199   0.047922
[30]             5         3                    0.018932            0.204321         0.160463              0.859078         0.013624   0.045808
[60, 120]        5         3                    0.005327            0.027451         0.132579              0.733123         0.013308   0.042355
                           4                    0.007433            0.038662         2.160956             31.490351         0.013657   0.042772
                           5                    0.108947            1.337132         0.248560              3.009527         0.013793   0.042067
                           6                    0.016109            0.046187         0.988663              3.108388         0.012317   0.041869
[60]             5         3                    0.007066            0.027667         0.285685              0.914433         0.014927   0.045583

In [21]:
tbl['SPY_m/s'] = tbl['SPY_CAGR/retnStd', 'mean'] / tbl['SPY_CAGR/retnStd', 'std'] 
tbl['grp-SPY_m/s'] = tbl['grp(CAGR/retnStd)_mean/std', 'mean'] - tbl['SPY_m/s'] 
tbl.sort_values(by='grp-SPY_m/s', ascending=False, inplace=True)
# tbl.sort_values(by='days_lookbacks', ascending=False, inplace=True)
tbl

grp(CAGR/retnStd)_mean               grp(CAGR/retnStd)_mean/std             SPY_CAGR/retnStd              SPY_m/s grp-SPY_m/s
                                                          mean           std                       mean         std             mean        std                      
days_lookbacks   days_eval sym_freq_cnt                                                                                                                              
[30, 60, 120]    5         5                    0.018160            0.398843         3.912322            101.978124         0.016680   0.048727  0.342316    3.570006
[60, 120]        5         4                    0.007433            0.038662         2.160956             31.490351         0.013657   0.042772  0.319300    1.841656
[15, 30]         5         4                    0.006659            0.037435         1.376197             23.283218         0.015032   0.048113  0.312426    1.063771
[15, 30, 60]     5         5                    0.007569            0.040046         1.149887             18.025109         0.014831   0.047090  0.314955    0.834932
[60, 120]        5         6                    0.016109            0.046187         0.988663              3.108388         0.012317   0.041869  0.294193    0.694470
[30, 60, 120]    5         6                    0.008071            0.046032         1.071601              9.622734         0.018199   0.047922  0.379764    0.691837
[15, 30, 60]     5         4                    0.004900            0.033411         0.575942              8.536839         0.015292   0.047954  0.318882    0.257059
[15]             5         3                 2034.288770        41862.104913         0.200269              0.732661         0.802652  18.187234  0.044133    0.156137
[120]            5         3                    0.012459            0.029804         0.364854              1.217032         0.017158   0.050010  0.343103    0.021751
[15, 30]         5         5                    0.006582            0.039122         0.305097              2.552450         0.013390   0.046803  0.286088    0.019009
[15, 30, 60, ... 5         4                    0.003047            0.028861         0.169792              2.509705         0.009486   0.050949  0.186188   -0.016396
[60]             5         3                    0.007066            0.027667         0.285685              0.914433         0.014927   0.045583  0.327469   -0.041784
[60, 120]        5         5                    0.108947            1.337132         0.248560              3.009527         0.013793   0.042067  0.327875   -0.079315
[30]             5         3                    0.018932            0.204321         0.160463              0.859078         0.013624   0.045808  0.297422   -0.136959
[15, 30, 60]     5         6                    0.007009            0.043564         0.171435              2.710300         0.015794   0.048252  0.327321   -0.155886
[30, 60, 120]    5         4                    0.005190            0.028486         0.176843              1.127394         0.016957   0.049012  0.345973   -0.169131
                           3                    0.007151            0.027594         0.181124              1.796506         0.017200   0.049004  0.350991   -0.169867
[15, 30]         5         3                    0.006109            0.024416         0.142637              0.616667         0.015046   0.047532  0.316535   -0.173898
[60, 120]        5         3                    0.005327            0.027451         0.132579              0.733123         0.013308   0.042355  0.314200   -0.181621
[15, 30, 60, ... 5         5                    0.006333            0.052436        -0.001727              1.659893         0.009541   0.050911  0.187414   -0.189141
[15, 30, 60]     5         3                    0.006098            0.028129         0.119825              0.713035         0.015308   0.047964  0.319159   -0.199334
[15, 30, 60, ... 5         3                    0.004119

In [22]:
# # https://stackoverflow.com/questions/31674195/plot-normal-distribution-given-mean-and-sigma-python
# # loc is mean, scale is standard deviation
# import pylab
# import numpy as np
# from scipy.stats import norm
# # x = np.linspace(-10000,100000,1000)
# x = np.linspace(-40e+10,50e+10,1000)
# y = norm.pdf(x, loc=2.562777e+10, scale=1.036925e+11)    # loc = mean, scale = standard deviation
# # z = norm.pdf(x, loc=3.540615e+10, scale=1.194430e+11)    # for example
# # z1 = norm.pdf(x, loc=298.805901, scale=826.875749)    # for example
# # z1 = norm.pdf(x, loc=1.021825, scale=1.505096)    # for example
# pylab.plot(x,y, 'b')
# # pylab.plot(x,z, 'g')
# # pylab.plot(x,z1, 'r')
# pylab.show()

In [23]:
# # get values of _cols, where grp(CAGR/retnStd)_mean is max after filtering out inf
# _cols = ['grp(CAGR/retnStd)_mean', 'grp(CAGR/retnStd)_std', 'grp(CAGR/retnStd)_mean/std']
# # _df_no_inf = df.loc[df['grp(CAGR/retnStd)_mean'] != np.inf]  # df with filter out inf in column grp(CAGR/UI)_mean 
# # _idx = _df_no_inf['grp(CAGR/retnStd)_mean'].idxmax()  # index value of max in grp(CAGR/UI)_mean 
# _idx = df['grp(CAGR/retnStd)_mean'].idxmax()  # index value of max in grp(CAGR/UI)_mean 
# grp_inf_replacement = df.loc[[_idx], _cols].squeeze()  # convert df (only has 1 row) to series
# print(f'_idx: {_idx}')
# grp_inf_replacement


In [24]:
# # get values of _cols, where SPY_CAGR/retnStd is max after filtering out inf
# _cols = ['SPY_CAGR/retnStd']
# # _df_no_inf = df.loc[df['SPY_CAGR/retnStd'] != np.inf]  # df with filter out inf in column grp(CAGR/UI)_mean 
# _idx = df['SPY_CAGR/retnStd'].idxmax()  # index value of max in grp(CAGR/UI)_mean 
# SPY_inf_replacement = df.loc[[_idx], _cols].squeeze()  # convert df (only has 1 row) to series
# print(f'_idx: {_idx}')
# SPY_inf_replacement

In [25]:
# # # https://stackoverflow.com/questions/50773107/how-to-replace-infinite-value-with-maximum-value-of-a-pandas-column
# # replace inf in column grp(CAGR/UI)_mean
# df['grp(CAGR/UI)_mean'].replace(np.inf, grp_inf_replacement['grp(CAGR/UI)_mean'], inplace=True)
# # replace NaN in column grp(CAGR/UI)_std
# df['grp(CAGR/UI)_std'].replace(np.nan, grp_inf_replacement['grp(CAGR/UI)_std'], inplace=True)
# # replace NaN in column grp(CAGR/UI)_mean/std
# df['grp(CAGR/UI)_mean/std'].replace(np.nan, grp_inf_replacement['grp(CAGR/UI)_mean/std'], inplace=True)
# # replace inf in column SPY_CAGR/UI
# df['SPY_CAGR/UI'].replace(np.inf, SPY_inf_replacement, inplace=True)
# df
# df

In [26]:
# print(f'n_samples:             {n_samples:>13}')
# print(f'days_lookbacks:        {days_lookbacks}')
# print(f'days_eval:             {days_eval:>13}')
# print(f'n_top_syms:            {n_top_syms:>13}')
# print(f'syms_start:            {syms_start:>13}')
# print(f'syms_end:              {syms_end:>13}')
# print(f'grp(CAGR/UI)_mean:     {grp_CAGR_d_UI[0]:>13,.3f}')
# print(f'grp(CAGR/UI)_std:      {grp_CAGR_d_UI[1]:>13,.3f}')
# print(f'grp(CAGR/UI)_mean/std: {grp_CAGR_d_UI[2]:>13,.3f}')
# print(f'SPY_CAGR/UI:           {SPY_CAGR_d_UI[0]:>13,.3f}')